In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# загружаем датафрейм и проводим чистку данных
df_diabets= pd.read_csv('/content/drive/MyDrive/diabetes.csv', delimiter= ',')

In [ ]:
df_diabets

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
df_diabets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
df_diabets.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df_diabets[df_diabets ==0].count()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [ ]:
df_diabets = df_diabets.drop(['SkinThickness', 'Insulin'], axis= 1)

In [ ]:
df_diabets.Glucose = df_diabets.Glucose.apply(lambda x: 117 if x == 0 else x)
df_diabets.BMI = df_diabets.BMI.apply(lambda x: 32 if x == 0 else x)
df_diabets.BloodPressure = df_diabets.BloodPressure.apply(lambda x: 70 if x == 0 else x)

In [ ]:
df_diabets[df_diabets ==0].count()

Pregnancies                 111
Glucose                       0
BloodPressure                 0
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [ ]:
# Разделяем предикторы и целевые значения
X = df_diabets.drop(['Outcome'], axis=1)
y = df_diabets['Outcome']

In [ ]:
# Делим выборку на тренировочный и тестовый набор данных
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
# Переводим данные в формат тензора
x_train = torch.tensor(list(x_train.values), dtype=torch.float32)
y_train = torch.tensor(list(y_train.values), dtype=torch.float32).reshape(-1,1)

In [ ]:
# Инициализируем модель
from torch.nn.modules.activation import ReLU
model_1 = nn.Sequential(
    nn.Linear(6, 4),
    nn.ELU(),
    nn.Linear(4, 3),
    nn.ReLU(),
    nn.Linear(3, 4),
    nn.ReLU(),
    nn.Linear(4,1),
    nn.Sigmoid()
)
print(model_1)

Sequential(
  (0): Linear(in_features=6, out_features=4, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=4, out_features=3, bias=True)
  (3): ReLU()
  (4): Linear(in_features=3, out_features=4, bias=True)
  (5): ReLU()
  (6): Linear(in_features=4, out_features=1, bias=True)
  (7): Sigmoid()
)


In [ ]:
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model_1.parameters(), lr=0.0001)

In [ ]:
# Обучаем модель на тренировочных данных
n_epochs = 500
batch_size = 30

for epoch in range(n_epochs):
  for i in range(0, len(x_train), batch_size):
    Xbatch = x_train[i: i+ batch_size]
    y_pred = model_1(Xbatch)
    ybatch = y_train[i:i+batch_size]
    loss = loss_fn(y_pred, ybatch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'Эпоха № {epoch+1}, значение функции потерь: {loss}')


Эпоха № 1, значение функции потерь: 0.26311925053596497
Эпоха № 2, значение функции потерь: 0.261913537979126
Эпоха № 3, значение функции потерь: 0.26094335317611694
Эпоха № 4, значение функции потерь: 0.2600954473018646
Эпоха № 5, значение функции потерь: 0.2594723403453827
Эпоха № 6, значение функции потерь: 0.25885435938835144
Эпоха № 7, значение функции потерь: 0.2582700252532959
Эпоха № 8, значение функции потерь: 0.25777488946914673
Эпоха № 9, значение функции потерь: 0.25734272599220276
Эпоха № 10, значение функции потерь: 0.2569887340068817
Эпоха № 11, значение функции потерь: 0.2566506564617157
Эпоха № 12, значение функции потерь: 0.2563338577747345
Эпоха № 13, значение функции потерь: 0.2559981048107147
Эпоха № 14, значение функции потерь: 0.2556585967540741
Эпоха № 15, значение функции потерь: 0.25531822443008423
Эпоха № 16, значение функции потерь: 0.254973828792572
Эпоха № 17, значение функции потерь: 0.25462308526039124
Эпоха № 18, значение функции потерь: 0.2542660534381

In [ ]:
x_test = torch.tensor(list(x_test.values), dtype=torch.float32)
y_test = torch.tensor(list(y_test.values), dtype=torch.float32).reshape(-1,1)

In [ ]:
# Проверяем модель на тестовых данных и выводим точность предсказаний построенной модели
with torch.no_grad():
  y_pred= model_1(x_test)
  accuracy = (y_pred.round() == y_test).float().mean()
  print(f'Точность модели: {round(float(accuracy*100), 2)} %')

Точность модели: 71.43 %


In [ ]:
# Строим 2 модель с 4 внутренними слоями
model_2 = nn.Sequential(
    nn.Linear(6, 4),
    nn.ELU(),
    nn.Linear(4, 3),
    nn.ReLU(),
    nn.Linear(3,3),
    nn.ELU(),
    nn.Linear(3, 4),
    nn.ReLU(),
    nn.Linear(4,1),
    nn.Sigmoid()
)
print(model_2)

Sequential(
  (0): Linear(in_features=6, out_features=4, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=4, out_features=3, bias=True)
  (3): ReLU()
  (4): Linear(in_features=3, out_features=3, bias=True)
  (5): ELU(alpha=1.0)
  (6): Linear(in_features=3, out_features=4, bias=True)
  (7): ReLU()
  (8): Linear(in_features=4, out_features=1, bias=True)
  (9): Sigmoid()
)


In [ ]:
loss_fn_2 = nn.MSELoss()
optimizer_2 = optim.Adam(model_2.parameters(), lr=0.0001)

In [ ]:
n_epochs = 500
batch_size = 30

for epoch in range(n_epochs):
  for i in range(0, len(x_train), batch_size):
    Xbatch = x_train[i: i+ batch_size]
    y_pred = model_2(Xbatch)
    ybatch = y_train[i:i+batch_size]
    loss = loss_fn_2(y_pred, ybatch)
    optimizer_2.zero_grad()
    loss.backward()
    optimizer_2.step()
  print(f'Эпоха № {epoch+1}, значение функции потерь: {loss}')

Эпоха № 1, значение функции потерь: 0.3602728545665741
Эпоха № 2, значение функции потерь: 0.35404813289642334
Эпоха № 3, значение функции потерь: 0.3480222523212433
Эпоха № 4, значение функции потерь: 0.3422195613384247
Эпоха № 5, значение функции потерь: 0.3366560637950897
Эпоха № 6, значение функции потерь: 0.33134618401527405
Эпоха № 7, значение функции потерь: 0.3262960612773895
Эпоха № 8, значение функции потерь: 0.32150784134864807
Эпоха № 9, значение функции потерь: 0.31697946786880493
Эпоха № 10, значение функции потерь: 0.31270405650138855
Эпоха № 11, значение функции потерь: 0.30866724252700806
Эпоха № 12, значение функции потерь: 0.3048613965511322
Эпоха № 13, значение функции потерь: 0.3013024926185608
Эпоха № 14, значение функции потерь: 0.2979934811592102
Эпоха № 15, значение функции потерь: 0.29492104053497314
Эпоха № 16, значение функции потерь: 0.29206106066703796
Эпоха № 17, значение функции потерь: 0.289404034614563
Эпоха № 18, значение функции потерь: 0.28693383932

In [ ]:
# Проверяем модель на тестовых данных и выводим точность предсказаний построенной модели
with torch.no_grad():
  y_pred= model_2(x_test)
  accuracy = (y_pred.round() == y_test).float().mean()
  print(f'Точность модели: {round(float(accuracy*100), 2)} %')

Точность модели: 72.29 %


#Вывод:
По показателям точности предскаазаний 2 построенная модель с 4 внутренними слоями показала наилучший результат(72,29%)